# <center>Prediccion de género de una película en función de su sinopsis</center>

Se obtuvieron de Kaggle dos datasets que hacen referencia a peliculas extraidas de IMDb, conteniendo en el primer caso varios de los atributos de las mismas, y para el segundo dataset la sinopsis de cada pelicula.  

Lo que se intenta conseguir es generar un modelo que permita predecir el genero de la pelicula en funcion de su sinopsis.

---

## Tratamiento de los datasets

In [1]:
#Importamos librerias
import pandas as pd

In [2]:
#Cargamos los datasets originales
df1 = pd.read_csv('ds_imdb_movies.csv')
df2 = pd.read_csv('ds_mpst_full_data.csv')

In [3]:
#Revisamos head
df1.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0


In [4]:
#Revisamos head
df2.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [5]:
#Obtenemos un df final realizando un merge entre df1 y df2 y ademas eliminamos las columnas que no nos interesan
res = df1.merge(df2[['imdb_id','plot_synopsis']] , left_on='imdb_title_id', right_on='imdb_id') \
    .drop(['writer', 'production_company', 'imdb_id','budget', 'usa_gross_income', 'worlwide_gross_income', \
           'metascore', 'reviews_from_users', 'reviews_from_critics'], axis = 1)

In [6]:
#Nos quedamos unicamente con las peliculas desde 1980 hasta la actualidad
res = res.loc[res['year'] >= 1980]
res.reset_index(drop=True, inplace=True)
res.rename(columns={'imdb_title_id':'imdb_id'},inplace=True)

In [7]:
#Exportamos el dataset definitivo a CSV
res.to_csv('orig_movies.csv')

In [8]:
del res

---

### Normalizacion de los generos

In [9]:
df = pd.read_csv('orig_movies.csv')
del df['Unnamed: 0']

In [10]:
tmp = df['genre'].str.split(", ", expand = True) 
df['inf_genre'] = tmp[0]
df.inf_genre = df.inf_genre.str.lower()
df

,imdb_id,title,original_title,year,date_published,genre,duration,country,language,director,actors,description,avg_vote,votes,plot_synopsis,inf_genre
0,tt0035423,Kate & Leopold,Kate & Leopold,2001,2002-04-05,"Comedy, Fantasy, Romance",118,USA,"English, French",James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",An English Duke from 1876 is inadvertedly drag...,6.4,75298,"On 28 April 1876, Leopold, His Grace the 3rd D...",comedy
1,tt0073537,Double Exposure,Double Exposure,1982,1983-11-09,"Comedy, Crime, Drama",100,USA,English,William Byron Hillman,"Michael Callan, Joanna Pettet, James Stacy, Pa...",A photographer for a men's magazine is disturb...,4.9,535,"The Putnams, Roger ('Ian Buchanan') and Maria ...",comedy
2,tt0076709,Si wang ta,Si wang ta,1981,1981-03-21,"Action, Crime, Mystery",96,Hong Kong,Cantonese,"See-Yuen Ng, Sammo Kam-Bo Hung","Bruce Lee, Tae-jeong Kim, Jung-Lee Hwang, Roy ...",After Billy Lo is killed while seeking the mur...,5.2,2670,"After a recent amount of challenges, Billy Lo ...",action
3,tt0078349,Sekai meisaku dôwa: Hakuchô no mizûmi,Sekai meisaku dôwa: Hakuchô no mizûmi,1981,1981-03-14,"Animation, Adventure, Family",75,Japan,Japanese,Kimio Yabuki,"Keiko Takeshita, Tarô Shigaki, Asao Koike, Yôk...",A prince falls in love with a princess cursed ...,7.8,667,Below is a synopsis based on the 1895 libretto...,animation
4,tt0078749,Alien 2 - Sulla Terra,Alien 2 - Sulla Terra,1980,1980-04-11,"Adventure, Horror, Sci-Fi",92,Italy,"English, Italian","Ciro Ippolito, Biagio Proietti","Belinda Mayne, Mark Bodin, Roberto Barrese, Be...",A spaceship lands back on Earth after a failed...,3.7,1104,The commercial spacecraft Nostromo is on a ret...,adventure
5,tt0078806,The Attic,The Attic,1980,1980-10-01,"Drama, Thriller, Horror",101,USA,English,"George Edwards, Gary Graver","Carrie Snodgress, Ray Milland, Ruth Cox, Rosem...",A librarian devotes her life to caring for her...,5.7,583,Emma (Moss) has a strong aversion towards her ...,drama
6,tt0078880,Bloodrage,Bloodrage,1980,1981-07-12,"Crime, Horror, Thriller",78,USA,English,Joseph Zito,"James Johnson, Judith-Marie Bergan, Jerry McGe...",A sexually frustrated young man kills hookers.,4.9,238,"A young man named Richard visits Beverly, a lo...",crime
7,tt0078935,Cannibal Holocaust,Cannibal Holocaust,1980,1980-02-07,"Adventure, Horror",95,Italy,"English, Spanish, Italian",Ruggero Deodato,"Robert Kerman, Francesca Ciardi, Perry Pirkane...",During a rescue mission into the Amazon rainfo...,5.9,47342,"In New York City, a TV news reporter recounts ...",adventure
8,tt0079285,Saturn 3,Saturn 3,1980,1980-06-27,"Adventure, Horror, Sci-Fi",96,UK,English,"Stanley Donen, John Barry","Farrah Fawcett, Kirk Douglas, Harvey Keitel",Two lovers stationed at a remote base in the a...,5.2,7754,The film opens at a space station in the vicin...,adventure
9,tt0079891,Shao Lin si,Shao Lin si,1982,1982,"Action, Drama",95,"China, Hong Kong",Mandarin,Hsin-Yen Chang,"Jet Li, Hai Yu, Chenghui Yu, Lan Ding, Jianqia...","A young man, hounded by a psychopathic general...",7.0,3726,The film opens with the chief Shaolin Monks re...,action


In [11]:
df.loc[df['genre'].str.match('Adventure, Action'), 'inf_genre'] = 'action'
df.loc[df['genre'].str.match('Adventure, Biography'), 'inf_genre'] = 'drama'
df.loc[df['genre'].str.match('Adventure, Comedy'), 'inf_genre'] = 'comedy'
df.loc[df['genre'].str.match('Adventure, Crime'), 'inf_genre'] = 'crime'
df.loc[df['genre'].str.match('Adventure, Drama'), 'inf_genre'] = 'drama'
df.loc[df['genre'].str.match('Adventure, Family'), 'inf_genre'] = 'fantasy'
df.loc[df['genre'].str.match('Adventure, Fantasy'), 'inf_genre'] = 'fantasy'
df.loc[df['genre'].str.match('Adventure, History'), 'inf_genre'] = 'romance'
df.loc[df['genre'].str.match('Adventure, Horror'), 'inf_genre'] = 'horror'
df.loc[df['genre'].str.match('Adventure, Mystery'), 'inf_genre'] = 'fiction'
df.loc[df['genre'].str.match('Adventure, Sci-Fi'), 'inf_genre'] = 'fiction'
df.loc[df['genre'].str.match('Adventure, Thriller'), 'inf_genre'] = 'thriller'
df.loc[df['inf_genre'].str.match('adventure'), 'inf_genre'] = 'action'

In [12]:
df.loc[df['genre'].str.match('Animation, Action'), 'inf_genre'] = 'action'
df.loc[df['genre'].str.match('Animation, Adventure'), 'inf_genre'] = 'comedy'
df.loc[df['genre'].str.match('Animation, Biography'), 'inf_genre'] = 'drama'
df.loc[df['genre'].str.match('Animation, Comedy'), 'inf_genre'] = 'comedy'
df.loc[df['genre'].str.match('Animation, Crime'), 'inf_genre'] = 'crime'
df.loc[df['genre'].str.match('Animation, Drama'), 'inf_genre'] = 'drama'
df.loc[df['genre'].str.match('Animation, Family'), 'inf_genre'] = 'fantasy'
df.loc[df['genre'].str.match('Animation, Fantasy'), 'inf_genre'] = 'fantasy'
df.loc[df['genre'].str.match('Animation, Horror'), 'inf_genre'] = 'horror'
df.loc[df['inf_genre'].str.match('animation'), 'inf_genre'] = 'fantasy'

In [13]:
df.loc[df['genre'].str.match('Biography, Comedy'), 'inf_genre'] = 'comedy'
df.loc[df['genre'].str.match('Biography, Crime'), 'inf_genre'] = 'crime'
df.loc[df['genre'].str.match('Biography, Drama'), 'inf_genre'] = 'drama'
df.loc[df['genre'].str.match('Biography, History'), 'inf_genre'] = 'historical'
df.loc[df['inf_genre'].str.match('biography'), 'inf_genre'] = 'romance'

In [14]:
df.loc[df['genre'].str.match('Family, Animation'), 'inf_genre'] = 'comedy'
df.loc[df['genre'].str.match('Family, Fantasy'), 'inf_genre'] = 'fantasy'
df.loc[df['genre'].str.match('Family, Horror'), 'inf_genre'] = 'horror'
df.loc[df['inf_genre'].str.match('family'), 'inf_genre'] = 'fantasy'

In [15]:
df.loc[df['genre'].str.match('Music'), 'inf_genre'] = 'thriller'
df.loc[df['genre'].str.match('Musical, Comedy'), 'inf_genre'] = 'comedy'
df.loc[df['genre'].str.match('Musical, Drama'), 'inf_genre'] = 'drama'
df.loc[df['genre'].str.match('Musical, Romance'), 'inf_genre'] = 'romance'

In [16]:
df.loc[df['genre'].str.match('Mystery, Romance'), 'inf_genre'] = 'romance'
df.loc[df['genre'].str.match('Mystery, Sci-Fi'), 'inf_genre'] = 'fiction'
df.loc[df['genre'].str.match('Mystery, Thriller'), 'inf_genre'] = 'thriller'
df.loc[df['inf_genre'].str.match('mystery'), 'inf_genre'] = 'fiction'

In [17]:
df.loc[df['genre'].str.match('Sci-Fi'), 'inf_genre'] = 'fiction'

In [18]:
df.loc[df['genre'].str.match('War'), 'inf_genre'] = 'drama'

In [19]:
df.loc[df['inf_genre'].str.match('western'), 'inf_genre'] = 'action'

In [20]:
df.groupby('inf_genre').count()[['imdb_id']]

,imdb_id
inf_genre,
action,2128
comedy,2397
crime,746
drama,1946
fantasy,155
fiction,47
historical,1
horror,771
romance,29


In [23]:
#Ordenamos las columnas
cols = ['imdb_id', 'title', 'original_title', 'year', 'date_published', 'genre', 'inf_genre', 'duration', 'country',\
        'language', 'director', 'actors', 'description', 'plot_synopsis', 'avg_vote', 'votes']
df = df[cols]
df

,imdb_id,title,original_title,year,date_published,genre,inf_genre,duration,country,language,director,actors,description,plot_synopsis,avg_vote,votes
0,tt0035423,Kate & Leopold,Kate & Leopold,2001,2002-04-05,"Comedy, Fantasy, Romance",comedy,118,USA,"English, French",James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",An English Duke from 1876 is inadvertedly drag...,"On 28 April 1876, Leopold, His Grace the 3rd D...",6.4,75298
1,tt0073537,Double Exposure,Double Exposure,1982,1983-11-09,"Comedy, Crime, Drama",comedy,100,USA,English,William Byron Hillman,"Michael Callan, Joanna Pettet, James Stacy, Pa...",A photographer for a men's magazine is disturb...,"The Putnams, Roger ('Ian Buchanan') and Maria ...",4.9,535
2,tt0076709,Si wang ta,Si wang ta,1981,1981-03-21,"Action, Crime, Mystery",action,96,Hong Kong,Cantonese,"See-Yuen Ng, Sammo Kam-Bo Hung","Bruce Lee, Tae-jeong Kim, Jung-Lee Hwang, Roy ...",After Billy Lo is killed while seeking the mur...,"After a recent amount of challenges, Billy Lo ...",5.2,2670
3,tt0078349,Sekai meisaku dôwa: Hakuchô no mizûmi,Sekai meisaku dôwa: Hakuchô no mizûmi,1981,1981-03-14,"Animation, Adventure, Family",comedy,75,Japan,Japanese,Kimio Yabuki,"Keiko Takeshita, Tarô Shigaki, Asao Koike, Yôk...",A prince falls in love with a princess cursed ...,Below is a synopsis based on the 1895 libretto...,7.8,667
4,tt0078749,Alien 2 - Sulla Terra,Alien 2 - Sulla Terra,1980,1980-04-11,"Adventure, Horror, Sci-Fi",horror,92,Italy,"English, Italian","Ciro Ippolito, Biagio Proietti","Belinda Mayne, Mark Bodin, Roberto Barrese, Be...",A spaceship lands back on Earth after a failed...,The commercial spacecraft Nostromo is on a ret...,3.7,1104
5,tt0078806,The Attic,The Attic,1980,1980-10-01,"Drama, Thriller, Horror",drama,101,USA,English,"George Edwards, Gary Graver","Carrie Snodgress, Ray Milland, Ruth Cox, Rosem...",A librarian devotes her life to caring for her...,Emma (Moss) has a strong aversion towards her ...,5.7,583
6,tt0078880,Bloodrage,Bloodrage,1980,1981-07-12,"Crime, Horror, Thriller",crime,78,USA,English,Joseph Zito,"James Johnson, Judith-Marie Bergan, Jerry McGe...",A sexually frustrated young man kills hookers.,"A young man named Richard visits Beverly, a lo...",4.9,238
7,tt0078935,Cannibal Holocaust,Cannibal Holocaust,1980,1980-02-07,"Adventure, Horror",horror,95,Italy,"English, Spanish, Italian",Ruggero Deodato,"Robert Kerman, Francesca Ciardi, Perry Pirkane...",During a rescue mission into the Amazon rainfo...,"In New York City, a TV news reporter recounts ...",5.9,47342
8,tt0079285,Saturn 3,Saturn 3,1980,1980-06-27,"Adventure, Horror, Sci-Fi",horror,96,UK,English,"Stanley Donen, John Barry","Farrah Fawcett, Kirk Douglas, Harvey Keitel",Two lovers stationed at a remote base in the a...,The film opens at a space station in the vicin...,5.2,7754
9,tt0079891,Shao Lin si,Shao Lin si,1982,1982,"Action, Drama",action,95,"China, Hong Kong",Mandarin,Hsin-Yen Chang,"Jet Li, Hai Yu, Chenghui Yu, Lan Ding, Jianqia...","A young man, hounded by a psychopathic general...",The film opens with the chief Shaolin Monks re...,7.0,3726


In [24]:
#Exportamos el dataset definitivo a CSV
df.to_csv('orig_movies.csv')